In [206]:
# %% Cell 1
# 1️⃣ Import libraries
import pandas as pd
import mysql.connector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix, classification_report
import mpld3  # For HTML visualization


In [207]:
# %% Cell 2
# 2️⃣ Connect to MySQL and load data
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",          # Your MySQL password
    database="ojt_internship_db"  # Replace with your database name
)

query = """
SELECT s.StudentID, s.StudentFirstName, s.StudentMiddleName, s.StudentLastName,
       s.Suffix, s.Gender, s.StudentContactNo, s.StudentEmail,
       c.CourseName, d.DeptName, s.Section,
       se.Question1, se.Question2, se.Question3, se.Question4, se.Question5,
       se.Question6, se.Question7, se.Question8, se.Question9, se.Question10,
       se.TotalPoints, se.Rating AS `Equivalent Rating`
FROM student s
INNER JOIN course c ON s.CourseID = c.CourseID
INNER JOIN department d ON c.DepartmentID = d.DepartmentID
LEFT JOIN studentevaluations se ON s.StudentID = se.StudentID;
"""

df = pd.read_sql(query, conn)
conn.close()

print("First 5 rows of SQL data:")
print(df.head())


First 5 rows of SQL data:
  StudentID StudentFirstName StudentMiddleName StudentLastName Suffix  Gender  \
0  25-00015          Gabriel          Bernardo        De Jesus   None    Male   
1  25-00027            Erica               Lim          Ferrer   None  Female   
2  25-00046          Vincent         Fernandez          Manalo   None    Male   
3  25-00049            Paula            Garcia        Martinez   None  Female   
4  25-00028            James            Santos          Flores   None    Male   

  StudentContactNo               StudentEmail  \
0       9171234515  gabriel.dejesus@gmail.com   
1       9171234527     erica.ferrer@gmail.com   
2       9171234546   vincent.manalo@gmail.com   
3       9171234549   paula.martinez@gmail.com   
4       9171234528     james.flores@gmail.com   

                                      CourseName  \
0       Bachelor of Science in Civil Engineering   
1       Bachelor of Science in Civil Engineering   
2       Bachelor of Science in Civil

C:\Users\acer\AppData\Local\Temp\ipykernel_13504\900396870.py:23: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [208]:
# %% Cell 3
# 3️⃣ Clean and preprocess
df.drop_duplicates(inplace=True)
df['Equivalent Rating'].fillna('NoRating', inplace=True)

# Features and target
feature_cols = [f'Question{i}' for i in range(1, 11)]
X = df[feature_cols].fillna(0)
y = df['Equivalent Rating']

# Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Classes encoded:", le.classes_)


Classes encoded: ['Failed' 'Passed']


C:\Users\acer\AppData\Local\Temp\ipykernel_13504\2118677142.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [209]:
# %% Cell 4
# 4️⃣ Split data and train Logistic Regression
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [210]:
# %% Cell 5
# 5️⃣ Generate HTML visualizations for dashboard

# Distribution plot
plt.figure(figsize=(6,4))
sns.countplot(x='Equivalent Rating', data=df)
plt.title("Distribution of Passed vs Failed")
with open("visual_distribution.html", "w", encoding="utf-8") as f:
    f.write(mpld3.fig_to_html(plt.gcf()))
plt.close()

# Feature correlation heatmap
plt.figure(figsize=(10,8))
sns.heatmap(df[feature_cols].corr(), cmap='coolwarm', annot=False)
plt.title("Feature Correlation Matrix")
with open("visual_correlation.html", "w", encoding="utf-8") as f:
    f.write(mpld3.fig_to_html(plt.gcf()))
plt.close()

# Confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, display_labels=le.classes_, cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
with open("visual_confusion.html", "w", encoding="utf-8") as f:
    f.write(mpld3.fig_to_html(plt.gcf()))
plt.close()


In [211]:
# %% Cell 6
# 6️⃣ Generate metrics HTML
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

metrics_html = f"""
<h2>Model Metrics</h2>
<p><b>Accuracy:</b> {acc:.4f}</p>
<h3>Classification Report:</h3>
<pre>{report}</pre>
<h3>Confusion Matrix:</h3>
<pre>{conf_matrix}</pre>
"""

with open("visual_metrics.html", "w", encoding="utf-8") as f:
    f.write(metrics_html)
